In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
os.chdir('/content/drive/MyDrive/')

In [3]:
!pip install kneed

In [4]:
import numpy as np
import pandas as pd
import copy
import os
import timeit

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

from data import fetch_dataset
from util import move_sliding_window, num_params
from model import LSTMNet, GRUNet
from algorithm import fedlbs

pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True, floatmode='fixed')

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('running on gpu')

else:
    device = torch.device("cpu")

running on gpu


# Parameters

In [6]:
window_size = 90 # Define window_size period and split inputs/labels\
batch_size = 1024
label_col_index = 0
prct = 10

# seq_len = 90  # (timestamps)
hidden_dim = 256
n_layers = 2
lr = 0.001
output_dim = 1

#fed train params
num_local_epochs = 1
max_rounds = 100 #nb of total rounds for training
num_clients_per_round = 30 #number of clients to participate in training

# Read

In [10]:
dataframes = fetch_dataset("./cleaned_data")

Floor0
Floor1
Floor2
Floor3
Floor4
Floor10_p7
Floor11_p3
Floor12_p6
Floor13_p4
Floor14_p7
Floor15_p10
Floor16_p4
Floor17_p10
Floor18_p7
Floor19_p8
Floor20_p7
Floor21_p2
Floor22_p7
Floor23_p11
Floor24_p11
Floor25_p3
Floor26_p6
Floor27_p2
Floor28_p4
Floor29_p3
Floor30_p10
Floor31_p3
Floor32_p8
Floor33_p4
Floor34_p10
Floor6_p6
Floor7_p6
Floor8_p6
Floor9_p7


In [ ]:
n_clients = len(dataframes) #total number of clients to partition data into

In [12]:

for _, df in dataframes.items():
    print(df.columns)
    print(df.shape)

Index(['total_demand', 'z1_AC1(kW)', 'z1_AC2(kW)', 'z1_AC3(kW)', 'z1_AC4(kW)',
       'z1_Light(kW)', 'z1_Plug(kW)', 'z2_AC1(kW)', 'z2_Light(kW)',
       'z2_Plug(kW)', 'z3_Light(kW)', 'z3_Plug(kW)', 'z4_AC1(kW)',
       'z4_Light(kW)', 'z4_Plug(kW)', 'z5_AC1(kW)', 'z5_Light(kW)',
       'z5_Plug(kW)', 'Year', 'Month', 'Day', 'Hour', 'Minute'],
      dtype='object')
(72864, 23)
Index(['total_demand', 'z1_AC1(kW)', 'z1_AC2(kW)', 'z1_AC3(kW)', 'z1_AC4(kW)',
       'z1_Light(kW)', 'z1_Plug(kW)', 'z2_AC1(kW)', 'z2_Light(kW)',
       'z2_Plug(kW)', 'z3_Light(kW)', 'z3_Plug(kW)', 'z4_AC1(kW)',
       'z4_Light(kW)', 'z4_Plug(kW)', 'z5_AC1(kW)', 'z5_Light(kW)',
       'z5_Plug(kW)', 'Year', 'Month', 'Day', 'Hour', 'Minute'],
      dtype='object')
(72864, 23)
Index(['total_demand', 'z1_AC1(kW)', 'z1_AC2(kW)', 'z1_AC3(kW)', 'z1_AC4(kW)',
       'z1_Light(kW)', 'z1_Plug(kW)', 'z2_AC1(kW)', 'z2_Light(kW)',
       'z2_Plug(kW)', 'z3_Light(kW)', 'z3_Plug(kW)', 'z4_AC1(kW)',
       'z4_Light(kW)', '

# Build the training set

In [13]:
train_loader = []
test_loader = []
label_scalers = []
i= 0
for _, df in dataframes.items():
    inputs_cols_indices = range(0, df.shape[1])  # use (total_demand,Year,Month,Day,Hour,Minute) columns as features
    if i ==30:
        break
    #move the window
    inputs, labels = move_sliding_window(
        df.values,
        window_size,
        inputs_cols_indices=inputs_cols_indices,
        label_col_index=label_col_index
    )

    # Normalize the input data columns
    sc = MinMaxScaler()
    # Obtaining the scaler for the labels(usage data) so that output can be re-scaled to actual value during evaluation
    label_sc = MinMaxScaler()

    # Split data into train/test portions and combining all data into a single array
    test_portion = int(0.2 * len(inputs))

    train_x = sc.fit_transform(inputs[:-test_portion].reshape(-1, window_size * df.shape[1]))
    train_x = train_x.reshape(-1, window_size, df.shape[1])
    train_y = label_sc.fit_transform(labels[:-test_portion])

    test_x = sc.transform(inputs[-test_portion:].reshape(-1, window_size * df.shape[1]))
    test_x = test_x.reshape(-1, window_size, df.shape[1])
    test_y = label_sc.transform(labels[-test_portion:])

    # test_x.append(testx)
    # test_y.append(testy)
    label_scalers.append(label_sc)

    # pytorch data loaders
    train_data = TensorDataset(torch.from_numpy(train_x).to('cpu'), torch.from_numpy(train_y).to('cpu'))
    train_loader.append(DataLoader(train_data, batch_size=batch_size, drop_last=True))# Drop the last incomplete batch
    test_data = TensorDataset(torch.from_numpy(test_x).to('cpu'), torch.from_numpy(test_y).to('cpu'))
    test_loader.append(DataLoader(test_data, batch_size=batch_size))# Drop the last incomplete batch

    # release some memory
    del train_x, train_y, test_x, test_y, train_data, test_data
    i=i+1

input_dim = next(iter(train_loader[0]))[0].shape[2]  # 22

(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)
(72774, 90, 23) (72774, 1)


# LSTM model

In [ ]:
lstm = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
model_type = 'LSTM'
print(lstm)
print(num_params(lstm))

## S =3

In [15]:
%%time
S = 3
lstm_K5_lbs = copy.deepcopy(lstm)
outputs_lbs, targets_lbs, loss_lbs, smape_lbs, mae_lbs, rmse_lbs = fedlbs(
    global_model = lstm_K5_lbs,
    client_train_loader = train_loader,
    test_loader = test_loader,
    label_sc = label_scalers,
    n_clients = n_clients,
    num_clients_per_round = num_clients_per_round,
    batch_size = batch_size,
    num_local_epochs = num_local_epochs,
    lr = lr,
    max_rounds = max_rounds,
    model_type = model_type,
    device = device,
    S=S
)

Streaming output truncated to the last 5000 lines.
round 35, starting client 27/30, id: 26
Epoch [1/1], Train Loss: 0.0015331825727896234
updating loss and weight
round 35, starting client 28/30, id: 27
Epoch [1/1], Train Loss: 0.0016144515518265374
updating loss and weight
round 35, starting client 29/30, id: 28
Epoch [1/1], Train Loss: 0.0011002250227777816
updating loss and weight
round 35, starting client 30/30, id: 29
Epoch [1/1], Train Loss: 0.008638892131524956
updating loss and weight
calc smape: 29.177525695662816%
MAE: 7.656785507094214
RMSE: 10.9284163929901
Average Loss:  0.004732032919575785

starting swapping round 36
clients:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
round 36, starting client 1/30, id: 0
Epoch [1/1], Train Loss: 0.0007582514789256884
round 36, starting client 2/30, id: 1
Epoch [1/1], Train Loss: 0.0007202332340706821
round 36, starting client 3/30, id: 2
Epoch [1/1], Train Loss: 0.0008110605743110814
ro

In [16]:
np.save(f'metrics/fedlbs_outputs_s{S}_C{num_clients_per_round}-.npy', np.array(outputs_lbs, dtype=object))
np.save(f'metrics/fedlbs_targets_s{S}_C{num_clients_per_round}-.npy', np.array(targets_lbs, dtype=object))
np.save(f'metrics/fedlbs_loss_s{S}_C{num_clients_per_round}-.npy', loss_lbs)
np.save(f'metrics/fedlbs_smape_s{S}_C{num_clients_per_round}-.npy', smape_lbs)
np.save(f'metrics/fedlbs_mae_s{S}_C{num_clients_per_round}-.npy', mae_lbs)
np.save(f'metrics/fedlbs_rmse_s{S}_C{num_clients_per_round}-.npy', rmse_lbs)